In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
from torchtune.modules import RMSNorm

/home/yuvrajsingh/miniconda3/envs/unsloth_env/lib/python3.10/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [14]:
@dataclass
class ModelArgs:
    #Hyperparameters

    block_size = 512
    batch_size = 64
    embeddings_dims = 2048
    attn_dropout = 0.1
    no_of_heads = 32 #IMP needs to be thoroughly calculated
    dropout = 0.1
    epochs = 100
    max_lr = 3e-4
    no_of_decoder_layers = 32 #IMP needs to be thoroughly calculated
    weight_decay_optim = 0.1
    beta_1 = 0.9
    beta_2 = 0.95
    clip = 1.0
    device = 'cpu'

In [15]:
class RMENorm(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims
    ):
        self.rmsnorm_layer = RMSNorm(dim=embeddings_dims)
        
    def forward(x, self):
        
        x = self.rmsnorm_layer(x)
        return x
        

In [19]:
class RotaryEmbeddings(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        block_size: int = ModelArgs.batch_size
    ):
        
        self.matrix = torch.ones((block_size, embeddings_dims, embeddings_dims), device=ModelArgs.device, requires_grad=False)
        self.theta = 0   
        for pos in range(block_size):
            for j in range(1, embeddings_dims // 2):
                self.theta = 10000 ** (-2*(pos-1) / embeddings_dims)
                self.matrix[pos, 2*j + 1, 2*j + 1] = torch.cos((pos* self.theta))
                self.matrix[pos, 2*j + 1, j + 1] = -torch.sin((pos* self.theta))
                self.matrix[pos, 2*j , 2*j ] = -torch.cos((pos* self.theta))
                self.matrix[pos, 2*j + 1, 2*j + 1] = torch.sin((pos* self.theta))
      
    def forward(self, x):
        return self.matrix

In [22]:
class RotaryAttentionHead(nn.Module):
    def __init__(
        self,
        embeddings_dims: int = ModelArgs.embeddings_dims,
        no_of_heads: int = ModelArgs.no_of_heads,
        attn_dropout: int = ModelArgs.attn_dropout
    ):
        self.head_size = embeddings_dims / no_of_heads
        self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.key = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.value = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=ModelArgs.device, bias=False)
        self.rotary_matrix = RotaryEmbeddings(embeddings_dims=embeddings_dims)
        self.dropout = nn.Dropout(p = ModelArgs.attn_dropout)
        
    def forward(self,x):
        query = self.query(x)
        key = self.key(x)
        values = self.value(x)
        
        batch, block_size, embeddings_dims = x.shape
        masked = torch.tril(torch.ones((block_size, block_size), device=ModelArgs.device, requires_grad=False))
        rotary_query = self.rotary_matrix @ torch.transpose(query, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        rotary_key = self.rotary_matrix @ torch.transpose(key, dim0=1, dim1=0) # (B,T,C ) @ (B,T,C,C) -> (B,C,T)
        weights = rotary_query @ (torch.transpose(rotary_key, dim0=-2, dim1=-1))
        weights_masked = weights.masked_fill(masked == 0, float('-inf'))
        scaled_weights = weights_masked / (torch.sqrt(self.head_size))
        scaled_weights = F.softmax(scaled_weights, dim=-1)
        value = scaled_weights @ values
        out = self.dropout(value)
        return out

In [18]:
masked = torch.tril(torch.ones((ModelArgs.block_size, ModelArgs.block_size), device=ModelArgs.device, requires_grad=False))
masked.shape

torch.Size([512, 512])

In [ ]:
class KVCache:
    def __init__(
        self,
        embeddings_dims: int =  ModelArgs.embeddings_dims,
        block_size: int  = ModelArgs.block_size,
        no_of_decoder_layers: int =ModelArgs.no_of_decoder_layers
    ):
        self.head_size = embeddings_dims / no_of_decoder_layers
        self.k_cache = torch.ones((embeddings_dims, self.head_size), device=ModelArgs.device, requires_grad=False)
        self.v_cache = torch.ones((embeddings_dims, self.head_size), device=ModelArgs.device, requires_grad=False)
    def update(
        self,
        k: torch.tensor,
        v: torch.tensor
    ):
        self.k_cache[] = k
        self.v_cache = v
        
    def get(self):
        